In [2]:
import pandas as pd
import numpy as np
import tqdm

## Loading Dataset
Scrubbed dataset collected by filtering through NCBI datasets (see `scrubbed-viral-data/README.md` for more info).

In [6]:
data_path = '../scrubbed-viral-data/scrubbed_sequences.csv'
df = pd.read_csv(data_path)
seqs = df['Sequence'].to_numpy()

## Input Sequences
Here you should put in the gene or sequence you wish to find local alignments of.

In [ ]:
v = 'GTAAATCCTTTGAGAAGAAGAGTCTCT'

## Output
These cells run the Hirschberg local align of input sequence v against